## Preprocesamiento de datos
...El mismo de la actividad anterior

In [ ]:
import pandas as pd
import numpy as np
import keras

In [ ]:
df = pd.read_csv("/content/sample_data/IMDB Dataset.csv")
print(df.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [ ]:
review = []
sentences = list(df['review'])
for sen in sentences:
  review.append(sen)

In [ ]:
labels = df['sentiment']
labels = np.array(list(map(lambda x: 1 if x =="positive" else 0, labels)))

In [ ]:
from sklearn.model_selection import train_test_split
train_sentences, test_sentences, train_labels, test_labels = train_test_split(review, labels, test_size=0.2)

In [ ]:
vocab_size = 1000
max_length = 120
embedding_dim = 16
trunc_type = 'post'
oov_tok = "<OOV>"

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Initialize the tokenizer class
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

# Generate the word index dictionary for training sentences
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

# Generate and pad the training sequences
sequences = tokenizer.texts_to_sequences(train_sentences)
padded = pad_sequences(sequences, maxlen = max_length, truncating= trunc_type)

# Generate and pad the test sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen = max_length, truncating=trunc_type)

## 1. Construcción del modelo

In [ ]:
# Build the model
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.LSTM(64, return_sequences=True),
    keras.layers.LSTM(32),
    keras.layers.Dense(6, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
])

### 2. Compilación y resumen del modelo

In [ ]:
# Setup the training parameters
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()

### 3. Entremaniento del modelo

In [ ]:
num_epochs = 10

# Train the model
history = model.fit(padded,
                    train_labels,
                    epochs=num_epochs,
                    validation_data=(test_padded, test_labels))

### 4. Visualización del historial de entrenamiento

In [ ]:
pd.DataFrame(history.history).plot(grid=True)

### Preguntas
1. ¿Qué tipo de modelo de red neuronal se construye en el código?
2. ¿Cuál es la función de activación utilizada en la capa de salida y por qué se elige esta función para
este problema?
3. ¿Cuál es la diferencia entre una capa LSTM bidireccional y una capa LSTM unidireccional?
4. ¿Qué signican los parámetros return_sequences=True en la primera capa LSTM bidireccional?
5. ¿Qué función de pérdida se utiliza en el modelo y por qué es adecuada para problemas de
clasicación binaria?
6. ¿Qué métrica se utiliza para evaluar el rendimiento del modelo durante el entrenamiento?
7. ¿Por qué se utilizan dos capas LSTM bidireccionales en el modelo?
8. ¿Qué hace la capa de embedding en el modelo y por qué es necesaria?


### Ejercicios
1. Modica el número de unidades de memoria en las capas LSTM y observa cómo afecta el
rendimiento del modelo.
2. Cambia la función de activación de la capa densa a 'tanh' y compara los resultados.
3. Entrena el modelo durante más épocas y observa si mejora el rendimiento en el conjunto de
validación.
4. Experimenta con diferentes valores para los parámetros max_length y vocab_size y observa cómo
afectan el tamaño y la eciencia del modelo.
5. Agrega una capa de regularización, como Dropout, al modelo y observa si mejora la generalización
del modelo.
6. Cambia la arquitectura del modelo agregando más capas ocultas y observa cómo afecta la
capacidad de aprendizaje del modelo

